# BEE 4750 Sea Level Rise Linear Program

**Name**: floating sigmas



## Setup

below are the necessary packages. this code will
1. take in a value for flooding
2. output an optimal solution for the municipality.


In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/4750coding/SLR-LP-4750`


In [2]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview
The goal is to 

## Exercise (3 points)


| Flood protection Type | Price/unit | BUildings protected per  |
|:-------------:|:----------------------:|:--------------------:|
|        Dry Installation     |   15,000 per building    |         1          |
|       Wet installation      |   5,000    |         1          |
|       Dyke       |    10million perlength perheight  |                   |



objective - minimize buildings lost

variables - how much of each flood proofing to install (A B and C) ((will find real examples later))

constraints - cannot go over the budget



In [9]:
using JuMP
using HiGHS  # HiGHS solver for linear programming

# Import required packages
using JuMP
using HiGHS

# Define parameters
c2 = 15_000        # Cost per wet installation
c3 = 5_000_000    # Cost of 1 m-high dyke per km
d = 200_000       # Damage cost per unprotected house
h = 300           # Total number of houses
p = 100           # Number of houses protected per km of dyke
budget = 4e6        #VARIABLE budget
inputSLR = .5      #VARIABLE from cella
maxSLR = 1        #VARIABLE actuallyFindThisValue
houseDestructionPerc = (inputSLR/maxSLR) #use this percentage for amount of housing lost.
dykeHeight = 1      #assume installed dyke must be 1m

# Create a model with HiGHS optimizer
model = Model(HiGHS.Optimizer)

# Define decision variables
@variable(model, x2 >= 0, Int)  # Number of wet installations
@variable(model, x3 >= 0)       # length of dyke in km



# Define the objective function
#cost*amt for each thingy, adjust damage cost function based on how much damage the house takes
@objective(model, Min, 
     c2 * x2 + (c3 * x3) + #upfront cost
     houseDestructionPerc * d * (h - ( x2 + p * x3)) + #damage cost of untreated houses
     x2*.2*houseDestructionPerc * d #damage cost of wet treated houses
)

# Add constraints
@constraint(model, x2 + p * x3 <= h)  # Protection cannot exceed total houses
@constraint(model, c2*x2 + c3*x3 <= budget)  #town budget constraint


# Solve the model
optimize!(model)

# Check and display results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found:")
    println("Number of wet installations (x2): ", value(x2))
    println("Length of dyke (x3): ", value(x3), " meters")
    println("2100 Minimum cost: \$", objective_value(model))

    upfront_cost = c2 * value(x2) + c3 * value(x3) #find the cost in 2030 (itll prob be budget until budget is v high then level)
    println("2030 upfront cost: \$", upfront_cost)


else
    println("No optimal solution found. Status: ", termination_status(model))
end






Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+06]
  Cost   [6e+04, 5e+06]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 4e+06]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s

Solving MIP model with:
   2 rows
   2 cols (0 binary, 1 integer, 0 implied int., 1 continuous)
   4 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   8710000         inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      12700000
  Dual bound        12700000
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    12700000 (objective)
                    

NEXT
- find actual types of protection, how much they actually cost and how many 'building equivalents' they actually protect
- find municipality data for NYC for realistic values of budget etc.
- correctly convert the MC Flood value into the buildings lost value 'D' here

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

I worked with Cella Schnabel